### 1. 索引有哪些数据结构？

### 2. 索引有哪些类型？

#### 2.1 多列索引

#### 2.2 覆盖索引

#### 2.3 二级索引

### 3. MySQL 的逻辑架构？

### 4. 事务的特点？隔离级别？

### 5. MySQL 的事务是如何实现的？

### 6. MVCC 原理？

### 7. 主从复制原理？

### 8. 如何实现备份？

### 9. 如何查看SQL使用了哪些索引

### 10. SQL 如何优化？LIMIT 几万行怎么办？

### 11.  乐观锁，悲观锁，表锁，行锁

### 12. 分布式事务如何实现？